# Getting the Git

In [1]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import timeit
import seaborn as sns
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
import matplotlib as mpl
import time
import csv
import math
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from time import sleep
import sys
import pickle
import codecs
import threading
os.environ['PATH'] += ";C\Program Files\Git\\bin"

'''# ---------------- Old Toolbar Updated -------------------------#
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) 
'''

def progbar(i,j):
    sys.stdout.write('\r')
    sys.stdout.write("%.2f%%" % round((i / j) * 100,2))
    sys.stdout.flush()
    sleep(0.001)

# Opening the full data

# Fuzzy matching on firm names

I use the fuzzywuzzy package and run through each firm name, comparing it to the remaining 5.6 million in the dataset. 

Each of the firm names is compared against a master list which is stripped of any non-string types (although I'm not sure why any would be in there, there seem to be a few.) I also drop duplicates but, again, I don't think there should be too many in there, given all the work already conducted using Stata.

In [2]:
fileObject = open('rawrecspickle','rb')
uniquerecs = pickle.load(fileObject)

Confirmed that there are no duplicates in the list. 

Now we're off to the races. This is going to take a goddamned long time so smoke 'em if you got 'em. We go through every firm name in the full list of 5.6m and compare the string to all of the others using the fuzzywuzzy method "process". 

Whoa...too long. I'm going to try and run this on the cluster. No sense in gumming up my machine.

In [5]:
dictter = dict(zip(range(57),range(0,6 * 10 ** 6,10 ** 5)))
start = dictter[0] #dictter[int(sys.argv[1])]
end = dictter[0+1] #dictter[int(sys.argv[1]+1)]
print(start,end)
end = 1000

0 100000


In [3]:
def pairs(rec,uniques):
    return [rec,[x[0] for x in process.extractBests(rec,uniques,scorer=fuzz.token_set_ratio,score_cutoff=90)]]

In [7]:
test = pairs(uniquerecs[1],uniquerecs[:100])
print(test)
threads = []
for i in range(5):
    t = threading.Thread(target = pairs,args = (uniquerecs[i],uniquerecs[:100]))
    threads.append(t)
    t.start()
t

['!1T Family Dental', ['!1T Family Dental']]


<Thread(Thread-25, started 13720)>

In [6]:
pairings=[]
counter = 0

for rec in uniquerecs[start:end]:
    pairings.append([rec,[x[0] for x in process.extractBests(rec,uniquerecs[:1000],scorer=fuzz.token_set_ratio,score_cutoff=90)]])
    progbar(counter,(end-start)-1)
    counter += 1

100.00%

In [12]:
new = []
for j in range(len(pairings)):
    new.append([pairings[j][0],[i for i in pairings[j][1] if pairings[j][0] != i]])
mask = [new[i] for i in range(len(new)) if len(new[i][1]) > 0]

new2 = []
for j in range(len(mask)):
    new2.append([mask[j][0],[x[0] for x in process.extractBests(mask[j][0],mask[j][1],scorer = fuzz.token_sort_ratio,score_cutoff=85)]])
mask2 = [new2[i] for i in range(len(new2)) if len(new2[i][1]) > 0]
print(len(mask),len(mask2))

forsurvey=[]
for i in range(len(mask2)-1):
    if len(mask2[i][1]) == 1:
        j = i+1
        if [mask2[i][0],mask2[i][1][0]] != [mask2[j][1][0],mask2[j][0]]:
            forsurvey.append(mask2[i])
    else:
        forsurvey.append(mask2[i])
forsurvey[:20]

276 72


[['#1 CLEANERS', ['$1.25 CLEANERS']],
 ['#1 SUN TAN AND TRAVEL, INC.', ['#1 SUN TAN & TRAVEL INC.']],
 ['#2 LA NAILS', ['#2 LA NAIL']],
 ['#7 STEAK-N-TAKE RESTAURANT', ['#7 STEAK-N-TAKE RESTARUANT']],
 ['#9 AREA VOLUNTEER FIRE DEPARTMENT', ['#9 AREA VOL. FIRE DEPARTMENT']],
 ['$ 1.49 DRY CLEAN CITY # 9',
  ['$1 69 DRY CLEAN CITY',
   '$1.69 DRY CLEAN CITY',
   '$1.79 DRY CLEAN CITY',
   '$1.50 DRY CLEAN CITY']],
 ['$1 69 DRY CLEAN CITY',
  ['$1.69 DRY CLEAN CITY',
   '$1.79 DRY CLEAN CITY',
   '$ 1.49 DRY CLEAN CITY # 9']],
 ['$1.00 DOLLAR STORE', ['$1 DOLLAR STORE']],
 ['$1.25 CLEANERS', ['#1 CLEANERS', '$1.25 DRY-CLEANERS']],
 ['$1.25 DRY CLEAN CENTER, INC.', ['$1.25 DRY CLEAN SUPER CENTER']],
 ['$1.25 DRY CLEAN SUPER CENTER',
  ['$1.49 DRY CLEAN SUPER CENTER',
   '$1.25 DRY CLEANERS SUPER CENTE',
   '$1.25 DRY CLEAN CENTER, INC.']],
 ['$1.25 DRY-CLEANERS', ['$1.25 CLEANERS']],
 ['$1.49 DRY CLEAN SUPER CENTER', ['$1.25 DRY CLEAN SUPER CENTER']],
 ['$1.50 CLEANERS',
  ['$1.25 CLEANERS

# Write a .txt file in the format for importing to Qualtrics

The code below can be converted to illustrate the process by which I create the .txt files that are formatted to be imported into Qualtrics. I don't run the code here though because it would take a long time.

In [9]:
maxs = int(math.ceil(len(forsurvey) / 100))
print(maxs)
length = len(forsurvey)

start = 0
counter = 0

for z in range(1,maxs+1):
    end = min(z*100,length)
    submask = forsurvey[start:end]
    start = end+1
    survey_file = open("range%s_survey_test_%s" %(1,z) +".txt","w")
    survey_file.write("[[AdvancedFormat]] \n\n")
    for j in range(0,len(submask),10):
        survey_file.write("[[Block: Firm Names - %s" % counter + "]]\n")
        for k in range(j,min(j+10,len(submask))):
            survey_file.write(" [[Question:MC:MultipleAnswer:Vertical]] \n \
            %s" % submask[k][0].upper() + "\n\n")
            survey_file.write("  [[AdvancedChoices]] \n")
            for i in range(0,len(submask[k][1])):
                survey_file.write("    [[Choice: %s" % i +"]] \n \
                %s" % submask[k][1][i].upper() +'\n')
            survey_file.write("\n")
        progbar(counter,maxs*10)
        counter += 1
    survey_file.close()

1
50.00%

In [44]:
new = []
for j in range(len(pairings)):
    new.append([pairings[j][0],[i for i in pairings[j][1] if pairings[j][0] != i]])
mask = [new[i] for i in range(len(new)) if len(new[i][1]) > 0]
new[:20]


new2 = []
for j in range(len(mask)):
    new2.append([mask[j][0],[x[0] for x in process.extractBests(mask[j][0],mask[j][1],scorer = fuzz.token_sort_ratio,score_cutoff=85)]])
mask2 = [new2[i] for i in range(len(new2)) if len(new2[i][1]) > 0]
print(len(mask),len(mask2))

forsurvey=[]
for i in range(len(mask2)-1):
    if len(mask2[i][1]) == 1:
        j = i+1
        if [mask2[i][0],mask2[i][1][0]] != [mask2[j][1][0],mask2[j][0]]:
            forsurvey.append(mask2[i])
    else:
        forsurvey.append(mask2[i])
forsurvey[:20]

anchors = [forsurvey[x][0] for x in range(len(forsurvey))]
pairs=[]
for j in range(len(forsurvey)):
    pairs.append([x for x in forsurvey[j][1]])

forsurvey2 = []
for i in range(5): #len(forsurvey)):
    neg = [x for j,x in enumerate(anchors) if j != i]
    print(neg[:6])
    forsurvey2.append([x for x in anchors[i] if x in neg])
forsurvey2[:10]


276 72
['#1 SUN TAN AND TRAVEL, INC.', '#2 LA NAILS', '#7 STEAK-N-TAKE RESTAURANT', '#9 AREA VOLUNTEER FIRE DEPARTMENT', '$ 1.49 DRY CLEAN CITY # 9', '$1 69 DRY CLEAN CITY']
['#1 CLEANERS', '#2 LA NAILS', '#7 STEAK-N-TAKE RESTAURANT', '#9 AREA VOLUNTEER FIRE DEPARTMENT', '$ 1.49 DRY CLEAN CITY # 9', '$1 69 DRY CLEAN CITY']
['#1 CLEANERS', '#1 SUN TAN AND TRAVEL, INC.', '#7 STEAK-N-TAKE RESTAURANT', '#9 AREA VOLUNTEER FIRE DEPARTMENT', '$ 1.49 DRY CLEAN CITY # 9', '$1 69 DRY CLEAN CITY']
['#1 CLEANERS', '#1 SUN TAN AND TRAVEL, INC.', '#2 LA NAILS', '#9 AREA VOLUNTEER FIRE DEPARTMENT', '$ 1.49 DRY CLEAN CITY # 9', '$1 69 DRY CLEAN CITY']
['#1 CLEANERS', '#1 SUN TAN AND TRAVEL, INC.', '#2 LA NAILS', '#7 STEAK-N-TAKE RESTAURANT', '$ 1.49 DRY CLEAN CITY # 9', '$1 69 DRY CLEAN CITY']


[[], [], [], [], []]

In [11]:
crosswalk1 = []
bisids=[]
for i in range(len(forsurvey)):
    crosswalk1.append(forsurvey[i][0])
    bisids.append("bid"+str(i))
    for j in range(len(forsurvey[i][1])):
        crosswalk1.append(forsurvey[i][1][j])
        bisids.append("bid"+str(i))

crosswalk = pd.DataFrame({'cw1' : crosswalk1,'bid':bisids})
print(forsurvey[0])
print(forsurvey[0][1])
#print(mask[23][1][0][0])

#inds = []
#for x in range(len(forsurvey)):
#    for i in range(len(forsurvey[x][1])):
#         if mask[23][1][0] in forsurvey[x][1][i]:
#                inds.append(x)
#mask[inds[1]]
#inds

crosswalk

['#1 CLEANERS', ['$1.25 CLEANERS']]
['$1.25 CLEANERS']


,bid,cw1
0,bid0,#1 CLEANERS
1,bid0,$1.25 CLEANERS
2,bid1,"#1 SUN TAN AND TRAVEL, INC."
3,bid1,#1 SUN TAN & TRAVEL INC.
4,bid2,#2 LA NAILS
5,bid2,#2 LA NAIL
6,bid3,#7 STEAK-N-TAKE RESTAURANT
7,bid3,#7 STEAK-N-TAKE RESTARUANT
8,bid4,#9 AREA VOLUNTEER FIRE DEPARTMENT
9,bid4,#9 AREA VOL. FIRE DEPARTMENT


In [ ]:
args = 1
name = str('test%s' % args + '.csv')
header = ['bid','cw1']
crosswalk.to_csv(name,sep=',',columns=header,index=False)

In [ ]:
%%bash
git add fec-string-prep.ipynb
git status
git commit -m "Further tinkering with the algorithm"
git push
git status

In [ ]:
%%bash
git status